### **Cuda Programming Applications**

This mini-lab targets some hands-on implementations and more practice on cuda in common real-world recurring tasks. Moreover, we aim to compare the outcomes of our low-level implementations with the built-in functions in popular frameworks as Pytorch. We'll revisit how you can fool cuda by passing a 2D array (for easier indexing)! Then we'll go straight to implement our Conv3D kernel function!

#### **Passing 2D array to cuda**

As we already know, array memory representation in cuda is linear, making higher-dimensional arrays represented also in a 1D fashion. That's why we need to flatten a 2D matrix to pass it to cuda in a row-major representation, making indexing kind of tedious. Now the question that pops-up: Could we in any way pass a 2D-organised array to cuda to do a more convenient double indexing?

The answer is: Yes! Yet, this comes with some limitations. To pass a 2D array and carry out double indexing in cuda, this array has to be statically allocated by the CPU, not dynamically allocated, so you need to know the array dimensions at the compile time. This way, the compiler is aware about the width of the 2D array, and can do the linearization process on its own. Moreover, a statically allocated array will be represented in memory in a contiguous 1D fashion. In contrast, if you dynamically allocate your matrix, you may or may not gurantee that all elements of the 2D array are contiguous, depending on the allocation fashion.

    // Consider for example, allocating the array this way:

    int* arr[r];
    for (i = 0; i < r; i++)
        arr[i] = (float*)malloc(c * sizeof(float));

A call to malloc here does not necessarily gurantee that the allocated memory is just after its preceding ones. Such discontinuouty makes it hard for the compiler to carry out the linearization.

#### Now let's consider the following matrix addition example based on double indexing

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\engah\appdata\local\temp\pip-req-build-s4e014cu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10818 sha256=816627ae86d1d305767f0033cfa18846ce59a2b5d2270675cc645171fb34eba2
  Stored in directory: C:\Users\engah\AppData\Local\Temp\pip-ephem-wheel-cache-vlha5k0p\wheels\ef\1d\c6\f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter
Source files will be saved in "C:\Users\engah\AppData\Local\Temp\tmp9s_g7r7d".


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\engah\AppData\Local\Temp\pip-req-build-s4e014cu'


In [2]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#define N 1000
#define M 500
#define MAX_ERR 1e-3
__global__ void MatAdd(float A[N][M], float B[N][M], float C[N][M])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    // Note: the first indexing specifies the row (y-axis), the second one specifies the column (x-axis)
    C[j][i] = A[j][i] + B[j][i];
}

int main(){

     // statically allocate the matrices
     float a[N][M], b[N][M], c[N][M];

    // Initialize a, b
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < M; ++j) {
            a[i][j] = i * 1.1;
            b[i][j] = j * 1.1;
        }
    }

    // Allocate device memory
    float *d_A, *d_B, *d_C; // Device pointer for the 2D array

    cudaMalloc((void**)&d_A, sizeof(float) * N * M);
    cudaMalloc((void**)&d_B, sizeof(float) * N * M);
    cudaMalloc((void**)&d_C, sizeof(float) * N * M);

    // Transfer data from host to device memory
    cudaMemcpy(d_A, a, sizeof(float) * N * M, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, b, sizeof(float) * N * M, cudaMemcpyHostToDevice);

    dim3 ThreadsPerBlock(16, 16);

    // Note that M here specifies the number of columns (on the X-axis), while N specifies the rows
    dim3 GridSize ((M - 1) / ThreadsPerBlock.x + 1, (N - 1) / ThreadsPerBlock.y + 1);

    // Casting the single pointer to an array of pointers
    MatAdd<<<GridSize, ThreadsPerBlock>>>((float(*) [M])d_A, (float(*) [M])d_B, (float(*) [M])d_C);

    // Transfer data back to host memory
    cudaMemcpy(c, d_C, sizeof(float) * N * M, cudaMemcpyDeviceToHost);


    // Verification
    for(int i = 0; i < N; i++){
      for(int j = 0; j < M; j++){
         assert(fabs(c[i][j] - a[i][j] - b[i][j]) < MAX_ERR);
      }
    }
    printf("PASSED\n");

    // Deallocate device memory
     cudaFree(d_A);
     cudaFree(d_B);
     cudaFree(d_C);

    // No need to deallocate host memory
}

### **Requirement**

A) A cuda program is required to carry out a 3D convolution over RGB images and save the output ones, the program is given a path to a folder containing the input images and that of an output folder that should contain the outputs, respectively as command line arguments.

1.   kernel1: basic implementation (no tiling)
2.   kernel2: tiling where each block matches the input tile size.
3.   kernel3: tiling where each block matches the output tile size.

Notes:
*   Add necessary paddings so that the output image size is the same as that of the input one.

*   The kernel should be able to handle a batch of images at a time, the batch size is passed as the 3rd argument.

B) Implement the same program in python, using the built-in convolution functions in Pytorch.

C) Profile each program carefully and do sufficient experiments to compare between them and collect insightful results. Organise your results in a tabular form and prepare a comprehensive report explaining all of your findings. Also mention the impact of declaring the mask as constant in terms of execution time and elaborate on this in your report.

#### **Helpers**

This section contains some helpers that could be needed for the requirement. Check it frequently.

**Helper1**: Read RGB images in C

In [ ]:
# Fetch stb_image library

!git clone https://github.com/nothings/stb.git
!cp stb/stb_image.h /usr/local/include/

Cloning into 'stb'...
remote: Enumerating objects: 8031, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 8031 (delta 99), reused 104 (delta 78), pack-reused 7868
Receiving objects: 100% (8031/8031), 5.59 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (5324/5324), done.


In [ ]:
# Read the image dimensions and pixels

%%writefile read_image.c
#define STB_IMAGE_IMPLEMENTATION

#include <stdio.h>
#include "stb_image.h"

const size_t NUM_PIXELS_TO_PRINT = 10;

int main(void) {
    int width, height, comp;
    unsigned char *data = stbi_load("image.jpeg", &width, &height, &comp, 0);
    if (data) {
        printf("width = %d, height = %d, comp = %d (channels)\n", width, height, comp);
        for (size_t i = 0; i < NUM_PIXELS_TO_PRINT * comp; i++) {
            printf("%d%s", data[i], ((i + 1) % comp) ? " " : "\n");
        }
        printf("\n");
    }
    return 0;
}

Overwriting read_image.c


In [ ]:
!g++ read_image.c -o readImage.out
!./readImage.out

width = 989, height = 1280, comp = 3 (channels)
153 161 161
153 161 161
153 161 161
153 161 161
153 161 161
153 161 161
153 161 161
153 161 161
152 160 160
152 160 160

